# VectorStore/QA, learn more

## Colab-specific setup

Make sure you have a Database and get ready to upload the Secure Connect Bundle and supply the Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details. Remember you need a **custom Token** with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).

_Note: this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/langchain/qa-advanced/)._


In [ ]:
# install required dependencies
! pip install \
    "git+https://github.com/hemidactylus/langchain@updated-full-preview--lab#egg=langchain&subdirectory=libs/langchain" \
    "cassio>=0.1.1" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2"

You will likely be asked to "Restart the Runtime" at this time, as some dependencies
have been upgraded. **Please do restart the runtime now** for a smoother execution from this point onward.

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name (e.g. cassio_tutorials): ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
from getpass import getpass
ASTRA_DB_APPLICATION_TOKEN = getpass('Your Astra DB Token ("AstraCS:..."): ')

### Astra DB Secure Connect Bundle

Please upload the Secure Connect Bundle zipfile to connect to your Astra DB instance.

The Secure Connect Bundle is needed to establish a secure connection to the database.
Click [here](https://awesome-astra.github.io/docs/pages/astra/download-scb/#c-procedure) for instructions on how to download it from Astra DB.

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                "token",
                ASTRA_DB_APPLICATION_TOKEN,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unsupported CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unsupported CQL Session mode')

### LLM Provider

In the cell below you can choose between **GCP Vertex AI** or **OpenAI** for your LLM services.
(See [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for more details).

Make sure you set the `llmProvider` variable and supply the corresponding access secrets in the following cell.

In [ ]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI', 'Azure_OpenAI'


In [ ]:
from getpass import getpass
if llmProvider == 'OpenAI':
    apiSecret = getpass(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
elif llmProvider == 'Azure_OpenAI':
    # a few parameters must be input
    apiSecret = input(f'Your API Key for LLM provider "{llmProvider}": ')
    os.environ['AZURE_OPENAI_API_KEY'] = apiSecret
    apiBase = input('The "Base URL" for your models (e.g. "https://YOUR-RESOURCE-NAME.openai.azure.com"): ')
    os.environ['AZURE_OPENAI_API_BASE'] = apiBase
    apiLLMDepl = input('The name of your LLM Deployment: ')
    os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'] = apiLLMDepl
    apiLLMModel = input('The name of your LLM Model (e.g. "gpt-4"): ')
    os.environ['AZURE_OPENAI_LLM_MODEL'] = apiLLMModel
    apiEmbDepl = input('The name for your Embeddings Deployment: ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'] = apiEmbDepl
    apiEmbModel = input('The name of your Embedding Model (e.g. "text-embedding-ada-002"): ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'] = apiEmbModel

    # The following is probably not going to change for some time...
    os.environ['AZURE_OPENAI_API_VERSION'] = '2023-03-15-preview'
else:
    raise ValueError('Unknown/unsupported LLM Provider')

### Colab preamble completed

The following cells constitute the demo notebook proper.

# VectorStore/QA, learn more

_**NOTE:** this uses Cassandra's "Vector Search" capability.
Make sure you are connecting to a vector-enabled database for this demo._

In the previous Quickstart, you have created the index and at the same time added the corpus of text to it.

In most cases, these two operations happen at different times: besides, often new documents keep being ingested.

This notebook demonstrates further interactions you can have with a Cassandra Vector Store.

It is assumed you have run the "VectorStore/QA, Quickstart" notebook (so that the vector store is not empty)

In [1]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

The setup is similar to the one you saw:

In [2]:
from langchain.vectorstores.cassandra import Cassandra

In [3]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

Below is the logic to instantiate the LLM and embeddings of choice. We chose to leave it in the notebooks for clarity.

In [4]:
import os
# creation of the LLM resources


if llmProvider == 'GCP_VertexAI':
    from langchain.llms import VertexAI
    from langchain.embeddings import VertexAIEmbeddings
    llm = VertexAI()
    myEmbedding = VertexAIEmbeddings()
    print('LLM+embeddings from Vertex AI')
elif llmProvider == 'OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'open_ai'
    from langchain.llms import OpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = OpenAI(temperature=0)
    myEmbedding = OpenAIEmbeddings()
    print('LLM+embeddings from OpenAI')
elif llmProvider == 'Azure_OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'azure'
    os.environ['OPENAI_API_VERSION'] = os.environ['AZURE_OPENAI_API_VERSION']
    os.environ['OPENAI_API_BASE'] = os.environ['AZURE_OPENAI_API_BASE']
    os.environ['OPENAI_API_KEY'] = os.environ['AZURE_OPENAI_API_KEY']
    from langchain.llms import AzureOpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = AzureOpenAI(temperature=0, model_name=os.environ['AZURE_OPENAI_LLM_MODEL'],
                      engine=os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'])
    myEmbedding = OpenAIEmbeddings(model=os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'],
                                   deployment=os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'])
    print('LLM+embeddings from Azure OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM+embeddings from OpenAI


## Re-use an existing Vector Store

Creating this `Cassandra` vector store, it will re-connect with the existing data on DB.

In practice, you are loading an existing, pre-populated vector store for further usage.

_(make sure you are using the very same embedding function every time! In fact, this is why we have a separate table for each embedding function, i.e. for each `llmProvider`.)_

In [5]:
myCassandraVStore = Cassandra(
    embedding=myEmbedding,
    session=session,
    keyspace=keyspace,
    table_name='vs_test1_' + llmProvider,
)

You can then re-instantiate the `index` from the vector store with:

In [6]:
index = VectorStoreIndexWrapper(vectorstore=myCassandraVStore)

and use it as you saw in the quickstart:

In [7]:
query = "Who is Luchesi?"
index.query(query, llm=llm)

' Luchesi is a person who is known for having a critical turn and for not being able to tell Amontillado from Sherry.'

## Further usage of the vector store

These are some of the ways you can query the store:

In [8]:
myCassandraVStore.similarity_search_with_score(
    "Does anyone have a coughing fit?",
    k=1,
)

[(Document(page_content='"Nitre," I replied.  "How long have you had that cough?"\n\n"Ugh! ugh! ugh!--ugh! ugh! ugh!--ugh! ugh! ugh!--ugh! ugh! ugh!--ugh!\nugh! ugh!"\n\nMy poor friend found it impossible to reply for many minutes.\n\n"It is nothing," he said, at last.', metadata={'source': 'texts/amontillado.txt'}),
  0.9052722957243504)]

### Adding new documents

Start with a very off-topic question, to demonstrate that no relevant documents are found (yet).

_Note: depending on the embedding function, you might still see some results, off-topic in practice, being found at this stage. In a full end-to-end QA session, however, these would likely be discarded by the LLM, which would presumably end up saying, "I don't know"._

In [9]:
SPIDER_QUESTION = 'Compare Agelenidae and Lycosidae'
myCassandraVStore.similarity_search_with_relevance_scores(
    SPIDER_QUESTION,
    k=1,
    score_threshold=0.8,
)

[(Document(page_content='"A huge human foot d\'or, in a field azure; the foot crushes a serpent\nrampant whose fangs are imbedded in the heel."\n\n"And the motto?"\n\n"_Nemo me impune lacessit_."\n\n"Good!" he said.', metadata={'source': 'texts/amontillado.txt'}),
  0.8635470113992079)]

You can add a couple of relevant paragraphs to the index, using the `add_texts` primitive:

In [10]:
spiderFacts = [
    """
    The Agelenidae are a large family of spiders in the suborder Araneomorphae.
    The body length of the smallest Agelenidae spiders are about 4 mm (0.16 in), excluding the legs,
    while the larger species grow to 20 mm (0.79 in) long. Some exceptionally large species,
    such as Eratigena atrica, may reach 5 to 10 cm (2.0 to 3.9 in) in total leg span.
    Agelenids have eight eyes in two horizontal rows of four. Their cephalothoraces narrow
    somewhat towards the front where the eyes are. Their abdomens are more or less oval, usually
    patterned with two rows of lines and spots. Some species have longitudinal lines on the dorsal
    surface of the cephalothorax, whereas other species do not; for example, the hobo spider does not,
    which assists in informally distinguishing it from similar-looking species.
    """,
    """
    Jumping spiders are a group of spiders that constitute the family Salticidae.
    As of 2019, this family contained over 600 described genera and over 6,000 described species,
    making it the largest family of spiders at 13% of all species.
    Jumping spiders have some of the best vision among arthropods and use it
    in courtship, hunting, and navigation.
    Although they normally move unobtrusively and fairly slowly,
    most species are capable of very agile jumps, notably when hunting,
    but sometimes in response to sudden threats or crossing long gaps.
    Both their book lungs and tracheal system are well-developed,
    and they use both systems (bimodal breathing).
    Jumping spiders are generally recognized by their eye pattern.
    All jumping spiders have four pairs of eyes, with the anterior median pair
    being particularly large. 
    """,
]
spiderMetadatas = [
    {'source': 'wikipedia/Agelenidae'},
    {'source': 'wikipedia/Salticidae'},
]

if llmProvider != 'Azure_OpenAI':
    ids = myCassandraVStore.add_texts(
        spiderFacts,
        spiderMetadatas,
    )
    print('\n'.join(ids))
else:
    # Note: this is a temporary mitigation of an Azure OpenAI error with asking for
    #       multiple embedding in a single request, which would error with:
    #           "InvalidRequestError: Too many inputs. The max number of inputs is 1"
    for spFact, spMetadata in zip(spiderFacts, spiderMetadatas):
        thisId = myCassandraVStore.add_texts(
            [spFact],
            [spMetadata],
        )[0]
        print(thisId)

5b44a1115e3245198a132a42705694a8
f1353d23c55b478192e77453cfdcc30a


Another way is to add a text through LangChain's `Document` abstraction.

Note that, using one of LangChain's splitters, long input documents are made into (possibly overlapping) digestible chunks without much boilerplate:

In [11]:
mySplitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=120)

In [12]:
lycoText = """
Wolf spiders are members of the family Lycosidae.
They are robust and agile hunters with excellent eyesight.
They live mostly in solitude, hunt alone, and usually do not spin webs.
Some are opportunistic hunters, pouncing upon prey as they
find it or chasing it over short distances;
others wait for passing prey in or near the mouth of a burrow.

Wolf spiders resemble nursery web spiders (family Pisauridae),
but wolf spiders carry their egg sacs by attaching them to their spinnerets,
while the Pisauridae carry their egg sacs with their chelicerae and pedipalps.
Two of the wolf spider's eight eyes are large and prominent;
this distinguishes them from nursery web spiders,
whose eyes are all of roughly equal size.
This can also help distinguish them from the similar-looking grass spiders.
"""

lycoDocument = Document(
    page_content=lycoText,
    metadata={'source': 'wikipedia/Lycosidae'}
)

Use the splitter to "shred" the input document:

In [13]:
lycoDocs = mySplitter.transform_documents([lycoDocument])
lycoDocs

[Document(page_content='Wolf spiders are members of the family Lycosidae.\nThey are robust and agile hunters with excellent eyesight.\nThey live mostly in solitude, hunt alone, and usually do not spin webs.\nSome are opportunistic hunters, pouncing upon prey as they', metadata={'source': 'wikipedia/Lycosidae'}),
 Document(page_content='Some are opportunistic hunters, pouncing upon prey as they\nfind it or chasing it over short distances;\nothers wait for passing prey in or near the mouth of a burrow.', metadata={'source': 'wikipedia/Lycosidae'}),
 Document(page_content='Wolf spiders resemble nursery web spiders (family Pisauridae),\nbut wolf spiders carry their egg sacs by attaching them to their spinnerets,\nwhile the Pisauridae carry their egg sacs with their chelicerae and pedipalps.', metadata={'source': 'wikipedia/Lycosidae'}),
 Document(page_content="while the Pisauridae carry their egg sacs with their chelicerae and pedipalps.\nTwo of the wolf spider's eight eyes are large and p

These are ready to be added to the index:

In [14]:
if llmProvider != 'Azure_OpenAI':
    ids = myCassandraVStore.add_documents(lycoDocs)
    print('\n'.join(ids))
else:
    # Note: this is a temporary mitigation of an Azure OpenAI error with asking for
    #       multiple embedding in a single request, which would error with:
    #           "InvalidRequestError: Too many inputs. The max number of inputs is 1"
    for lycoDoc in lycoDocs:
        thisId = myCassandraVStore.add_documents([lycoDoc])[0]
        print(thisId)

5b7d726f444b421f9f25a55d73291766
4ab50d78b1664ad094e9c8b95b24ca44
e9f81b5662dc409db259547348fd099b
f2d60ab62b75478fb5d548c29b239529
e6ebc3f66acb4c538d6d45fc8d05d6c8


### Querying the store again

Time to repeat the question:

In [15]:
myCassandraVStore.similarity_search_with_relevance_scores(
    SPIDER_QUESTION,
    k=3,
    score_threshold=0.8,
)

[(Document(page_content='\n    The Agelenidae are a large family of spiders in the suborder Araneomorphae.\n    The body length of the smallest Agelenidae spiders are about 4 mm (0.16 in), excluding the legs,\n    while the larger species grow to 20 mm (0.79 in) long. Some exceptionally large species,\n    such as Eratigena atrica, may reach 5 to 10 cm (2.0 to 3.9 in) in total leg span.\n    Agelenids have eight eyes in two horizontal rows of four. Their cephalothoraces narrow\n    somewhat towards the front where the eyes are. Their abdomens are more or less oval, usually\n    patterned with two rows of lines and spots. Some species have longitudinal lines on the dorsal\n    surface of the cephalothorax, whereas other species do not; for example, the hobo spider does not,\n    which assists in informally distinguishing it from similar-looking species.\n    ', metadata={'source': 'wikipedia/Agelenidae'}),
  0.9029937548261041),
 (Document(page_content="while the Pisauridae carry their 

### Item removal and expiration

#### Time-To-Live (TTL)

If you provide a TTL value when creating the store, every entry will expire away a certain time after its insertion:

In [16]:
myCassandraVStoreWithTTL = Cassandra(
    embedding=myEmbedding,
    session=session,
    keyspace=keyspace,
    table_name='vs_test1_shortlived_' + llmProvider,
    ttl_seconds=120,
)

The following two documents will be available for two minutes.

In [17]:
if llmProvider != 'Azure_OpenAI':
    ids = myCassandraVStoreWithTTL.add_documents(lycoDocs[0:2])
    print('\n'.join(ids))
else:
    # Note: this is a temporary mitigation of an Azure OpenAI error with asking for
    #       multiple embedding in a single request, which would error with:
    #           "InvalidRequestError: Too many inputs. The max number of inputs is 1"
    for lycoDoc in lycoDocs[0:2]:
        thisId = myCassandraVStoreWithTTL.add_documents([lycoDoc])[0]
        print(thisId)

4a67cdc623c54244bb7931a8575610c4
9f59ccbd9e694d7c8a74106b9e3434de


Alternatively, for a finer control of the time-to-live, you can specify it at insertion time -- which would anyway have precedence over the store-level definition. So, these documents will survive for twenty seconds:

In [18]:
if llmProvider != 'Azure_OpenAI':
    ids = myCassandraVStore.add_documents(lycoDocs[2:], ttl_seconds=20)
    print('\n'.join(ids))
else:
    # Note: this is a temporary mitigation of an Azure OpenAI error with asking for
    #       multiple embedding in a single request, which would error with:
    #           "InvalidRequestError: Too many inputs. The max number of inputs is 1"
    for lycoDoc in lycoDocs[2:]:
        thisId = myCassandraVStore.add_documents([lycoDoc], ttl_seconds=20)[0]
        print(thisId)

cfc09a38e082407daaf025f0cf067487
1daee1e823fb464dbbab28177b789a68
aea25de5b3b640439168485a77bdd08b


#### Manual removal of entries

You can delete individual documents from the store.

However, you first need to retrieve their identifier with a similarity search. The following method returns a list of matching 3-tuples, whose last item is the id of the document:

In [19]:
spiderDocIds = []
for doc, score, docId in myCassandraVStore.similarity_search_with_score_id('Compare Agelenidae and Lycosidae'):
    print(f'  * [{score:.3f}] "{doc.page_content[:32].strip()}..." ({docId})')
    spiderDocIds.append(docId)

  * [0.903] "The Agelenidae are a large..." (5b44a1115e3245198a132a42705694a8)
  * [0.901] "while the Pisauridae carry their..." (f2d60ab62b75478fb5d548c29b239529)
  * [0.901] "while the Pisauridae carry their..." (1daee1e823fb464dbbab28177b789a68)
  * [0.893] "Wolf spiders resemble nursery we..." (e9f81b5662dc409db259547348fd099b)


At this point you can perform the actual document deletion:

In [20]:
for spiderDocId in spiderDocIds:
    myCassandraVStore.delete_by_document_id(spiderDocId)

The last method to remove entries from the store is demonstrated next.

### Cleanup

You're done.

In order to leave the index empty for the next demo run, you may want to clean the index (i.e. empty the table on DB).

Just don't take this operation lightly in production!

In [21]:
myCassandraVStore.clear()

## What now?

This demo is hosted [here](https://cassio.org/frameworks/langchain/qa-advanced/) at cassio.org.

Discover the other ways you can integrate 
Cassandra/Astra DB with your ML/GenAI needs,
right **within [your favorite framework](https://cassio.org/frameworks/langchain/about/)**.